In [71]:
#Business Question:
#What is the quantum of price increase for every SKU in Year 2022 for revenue optimization and what is the value upside to business?
#ie how sensitive is demand if business increases the price? Will this increase the sales?
#ie how senstivie is demand if business reduces the price? How much sales will increase?
#the above will be quantified using Price Elasticity model with Linear Regression to determine the max price for revenue optimization.

#We would use following python libraries:
#pandas
#numpy
#matplotlib
#statsmodels
#linearRegression
#inventorize

#We would solve the Business Questio using OSEMN framework:
#Obtain = Dataset is collected from Kaggle, how dataset is determined is by ensuring there is historical price and quantity sold
#data, the price need at least to have more than one price variations with the respective quantity sold in order to be able
#to observe the sales demand reaction towards the product price changes.

#Scrub = Clean data, ensuring price & quantity are in integer form, product description is in object form or conversion is needed.

#Explore = find the pattern using statistical methods

#Model = Construct models to predict and forecast

#Interpret = Put the results into good use, ensure stakeholders understand in layman terms.

#let's get started:

#we need to load libraries:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import inventorize as inv
import seaborn as sns

#Obtain (dataset is obtained from Kaggle)
retail_clean=pd.read_csv(r'C:\Users\user\Desktop\Presentation\retail_clean.csv')


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
#Scrub
#let's look at the data to ensure the file is loading correctly with 3 samples choosen
retail_clean.sample(3)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
923717,572921,23555,LANDMARK FRAME NOTTING HILL,1,2011-10-26T16:57:00Z,12.50,15993.0,United Kingdom
309645,520494,22328,ROUND SNACK BOXES SET OF 4 FRUITS,48,2010-08-26T12:09:00Z,2.55,12621.0,Germany
901699,571257,85176,SEWING SUSAN 21 NEEDLE SET,1,2011-10-14T17:17:00Z,1.63,NaN,United Kingdom


In [73]:
#Scrub
#to check the format of Decription, Quantity & Price
retail_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042721 entries, 0 to 1042720
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1042721 non-null  object 
 1   StockCode    1042721 non-null  object 
 2   Description  1042721 non-null  object 
 3   Quantity     1042721 non-null  int64  
 4   InvoiceDate  1042721 non-null  object 
 5   Price        1042721 non-null  float64
 6   Customer ID  805618 non-null   float64
 7   Country      1042721 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 63.6+ MB


In [74]:
#Scrub
#to check how many SKUs under Description
retail_clean['Description'].unique()

array(['15CM CHRISTMAS GLASS BALL 20 LIGHTS', 'PINK CHERRY LIGHTS',
       'WHITE CHERRY LIGHTS', ..., 'SET 10 CARDS SNOWY ROBIN 17099',
       'SET 10 CARDS SWIRLY XMAS TREE 17104',
       'CREAM HANGING HEART T-LIGHT HOLDER'], dtype=object)

In [75]:
#Scrub
#as the above consisted a long list of SKUs, do a quick count on the number of SKUs
len(retail_clean['Description'].unique())

5425

In [76]:
#Choose one country platform 
retail_clean = retail_clean[retail_clean['Country']=='United Kingdom']

#Check values (highest invoice count) in different SKUs under Description for price elasticity assessment
description = retail_clean['Description'].value_counts()
print("United Kingdom Percentage\n{}\n{}\n United Kingdom Description values count\n{}\n{}".format(
      0*"-", (description / len(retail_clean.index))[:5] * 100, 0*"-", description[:5]))


United Kingdom Percentage

WHITE HANGING HEART T-LIGHT HOLDER    0.580913
REGENCY CAKESTAND 3 TIER              0.372789
JUMBO BAG RED RETROSPOT               0.330059
ASSORTED COLOUR BIRD ORNAMENT         0.289623
PARTY BUNTING                         0.270968
Name: Description, dtype: float64
 United Kingdom Description values count

WHITE HANGING HEART T-LIGHT HOLDER    5574
REGENCY CAKESTAND 3 TIER              3577
JUMBO BAG RED RETROSPOT               3167
ASSORTED COLOUR BIRD ORNAMENT         2779
PARTY BUNTING                         2600
Name: Description, dtype: int64


In [77]:
#As per see, White Hanging Heart T-Light Holder is the biggest description.
#For this reason in this occasion, we would analyze the price elasticities of this description.

#Select White Hanging Heart T-Light Holder
df_white = retail_clean[retail_clean['Description']=='WHITE HANGING HEART T-LIGHT HOLDER']

#use Groupby and Aggregate to group
df_white = df_white.groupby(['Description']).agg(total_sales=('Quantity', 'sum'), price = ('Price','mean')).reset_index()

#White Hanging Heart T-Light Holder data used for price elasticities
df_white.head()

,Description,total_sales,price
0,WHITE HANGING HEART T-LIGHT HOLDER,89012,3.142734


In [78]:
#to do linear elastiscity for every SKU, as there is 5425 sku, looping is doing for 1 SKU for time being so that easier to extend to few more if needed
#make a loop for every sku with a linear elasticity

keys = df_white.Description.unique()
print(keys)

['WHITE HANGING HEART T-LIGHT HOLDER']


In [101]:
import pandas as pd
keys = df_white.Description.unique()

empty_data=pd.DataFrame()
for key in keys:
    try:
        a=df_white[df_white.Description==key]
        cpst=0.4*max(a['price'])
        current_price=a['price'].mean()
        elasticity= inv.linear_elasticity(a['price'],a['total_sales'], current_price,cost)
        elasticty= {k:v[0] for k, v in elasticity.items()}
        data=pd.DataFrame(elasticity,index=[0])
        data['Description']=key
        empty_data=pd.concat([empty_data,data],axis=0)
    except:
            continue
            

C:\Users\user\anaconda3\lib\site-packages\inventorize\__init__.py:662: RuntimeWarning: divide by zero encountered in true_divide
  optimum_profit=(-derv_p*cost_of_product+ intercept)/(2*-derv_p)
C:\Users\user\anaconda3\lib\site-packages\inventorize\__init__.py:663: RuntimeWarning: divide by zero encountered in true_divide
  optimum_revenue =intercept/(2*-derv_p)


In [102]:
empty_data

""
